<a href="https://colab.research.google.com/github/Jalajagr83/ML_Playook/blob/main/Bert_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!pip install emoji
#!pip install simpletransformers
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from simpletransformers.classification import ClassificationModel
import sklearn
import itertools
import emoji

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/text_classification_with_BERT/train.csv')
df.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:

columns_drop = ['keyword','location']
df.drop(columns=columns_drop,inplace=True)
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
fake_tweets = df[df.target == 0]
fake_tweets.shape

(4342, 3)

In [8]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}


In [9]:

def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [10]:

df['text']=df['text'].apply(remove_contractions)
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [12]:
df['text'] =df['text'].apply(clean_dataset)
df.head()

,id,text,target
0,1,Deeds Reason earthquake ALLAH Forgive,1
1,4,Forest Ronge Canada,1
2,5,residents asked shelter place being notified o...,1
3,6,people receive wildfires evacuation orders Cal...,1
4,7,photo Alaska smoke wildfires pours school,1


In [13]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(df['text'], df['target'], test_size=0.20, random_state=42)

train_df_clean = pd.concat([X_train_clean, y_train_clean], axis=1)
print("Shape of training data set: ", train_df_clean.shape)
print("View of data set: ", train_df_clean.head())

Shape of training data set:  (6090, 2)
View of data set:                                                     text  target
4996  Courageous honest analysis Atomic Hiroshima70 ...       1
3263          shame became engulfed flames boycottBears       0
4907  rescind medals honor given soldiers Massacre W...       1
2855  Worried about drought might affect Extreme Wea...       1
4716                           BlastPower PantherAttack       0


In [14]:
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 2,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False,
    'wandb_project': "visualization-demo"
}

In [17]:

eval_df_clean = pd.concat([X_test_clean, y_test_clean], axis=1)
print("Shape of Eval data set: ", eval_df_clean.shape)

Shape of Eval data set:  (1523, 2)


In [15]:
model_BERT = ClassificationModel('bert', 'bert-base-cased', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [18]:
model_BERT.train_model(train_df_clean, eval_df=eval_df_clean)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/6090 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


...

Running Epoch 1 of 2:   0%|          | 0/191 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

(382,
 {'auprc': [0.7510892694812671,
   0.8528257932859971,
   0.8519244258305787,
   0.8552957184181711,
   0.8571096754963266],
  'auroc': [0.8237007824041918,
   0.8547607126612673,
   0.8558396476889282,
   0.85579381057991,
   0.8557373956765029],
  'eval_loss': [0.5451223130027453,
   0.4553169409434001,
   0.4719281954069932,
   0.4871426373720169,
   0.47408489013711613],
  'fn': [162, 180, 155, 173, 191],
  'fp': [175, 121, 158, 132, 112],
  'global_step': [100, 191, 200, 300, 382],
  'mcc': [0.5488246941919359,
   0.5930311000852466,
   0.5800525701556918,
   0.5880872957443343,
   0.590183757300275],
  'tn': [699, 753, 716, 742, 762],
  'tp': [487, 469, 494, 476, 458],
  'train_loss': [0.4384186267852783,
   0.46519652009010315,
   0.26981475949287415,
   0.6383641958236694,
   0.306252658367157]})

In [19]:

result, model_outputs, wrong_predictions = model_BERT.eval_model(eval_df_clean, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1523 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/24 [00:00<?, ?it/s]

Training loss,0.63836
lr,1e-05
global_step,382
_runtime,626
_timestamp,1617608549
_step,7
tp,458
tn,762
fp,112
fn,191
mcc,0.59018


Training loss,▄▁█
lr,█▅▁
global_step,▁▁▃▃▃▆▆█
_runtime,▁▁▃▄▄▆▆█
_timestamp,▁▁▃▄▄▆▆█
_step,▁▂▃▄▅▆▇█
tp,▇▃█▅▁
tn,▁▇▃▆█
fp,█▂▆▃▁
fn,▂▆▁▅█
mcc,▁█▆▇█


In [20]:

model_Roberta = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
model_Roberta.train_model(train_df_clean, eval_df=eval_df_clean)
result, model_outputs, wrong_predictions = model_Roberta.eval_model(eval_df_clean, acc=sklearn.metrics.accuracy_score)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/24 [00:00<?, ?it/s]

Training loss,0.47202
lr,1e-05
global_step,382
_runtime,655
_timestamp,1617609790
_step,7
tp,493
tn,740
fp,134
fn,156
mcc,0.60924


Training loss,█▁█
lr,█▅▁
global_step,▁▁▃▃▃▆▆█
_runtime,▁▁▃▄▄▆▆█
_timestamp,▁▁▃▄▄▆▆█
_step,▁▂▃▄▅▆▇█
tp,▄█▁▃█
tn,▁▄▆█▆
fp,█▅▃▁▃
fn,▅▁█▆▁
mcc,▁▆▄▇█


In [21]:
model_albert = ClassificationModel('albert', 'albert-base-v2', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
model_albert.train_model(train_df_clean, eval_df=eval_df_clean)
result, model_outputs, wrong_predictions = model_albert.eval_model(eval_df_clean, acc=sklearn.metrics.accuracy_score)


Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

  0%|          | 0/1523 [00:00<?, ?it/s]

  0%|          | 0/1523 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/24 [00:00<?, ?it/s]

Training loss,0.38022
lr,1e-05
global_step,382
_runtime,641
_timestamp,1617610651
_step,7
tp,441
tn,734
fp,140
fn,208
mcc,0.52869


Training loss,█▁▄
lr,█▅▁
global_step,▁▁▃▃▃▆▆█
_runtime,▁▁▃▄▄▆▆█
_timestamp,▁▁▃▄▄▆▆█
_step,▁▂▃▄▅▆▇█
tp,█▁▃▆▃
tn,▁██▅▇
fp,█▁▁▄▂
fn,▁█▆▃▆
mcc,▁▇█▆▇


In [22]:

test_df = pd.read_csv('https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/text_classification_with_BERT/test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [23]:
## Drop columns for keyword and location
columns_todrop = ['keyword','location']
test_df.drop(columns=columns_todrop,inplace=True)
### Change contractions
test_df['text']=test_df['text'].apply(remove_contractions)
## Clean Data set
test_df['text'] =test_df['text'].apply(clean_dataset)
test_df.head()

,id,text
0,0,happened terrible crash
1,2,Heard about earthquake different cities everyone
2,3,there forest geese fleeing across street cannot
3,9,Apocalypse lighting Spokane wildfires
4,11,Typhoon Soudelor kills China Taiwan


In [35]:
predictions, raw_outputs = model_Roberta.predict(xtest)

  0%|          | 0/3263 [00:00<?, ?it/s]

AssertionError: ignored

In [ ]:

test_df['target']=predictions
test_df.tail()

In [ ]:
test_df['target'].value_counts()

In [25]:
test_tweet1 = "#COVID19 will spread across U.S. in coming weeks. We’ll get past it, but must focus on limiting the epidemic, and preserving lif"
test_tweet1 = remove_contractions(test_tweet1)
test_tweet1 = clean_dataset(test_tweet1)

In [26]:
predictions, _ = model_Roberta.predict([test_tweet1])
response_dict = {0: 'Fake', 1: 'Real'}
print("Prediction is: ", response_dict[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Prediction is:  Real


In [27]:

test_tweet2 = "BREAKING: Confirmed flooding on NYSE. The trading floor is flooded under more than 3 feet of water."
test_tweet2 = remove_contractions(test_tweet2)
test_tweet2 = clean_dataset(test_tweet2)

predictions, _ = model_Roberta.predict([test_tweet2])
response_dict = {0: 'Fake', 1: 'Real'}
print("Prediction is: ", response_dict[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Prediction is:  Real


In [28]:

test_tweet3 = "Everything is ABLAZE. Please run!!"
test_tweet3 = remove_contractions(test_tweet3)
test_tweet3 = clean_dataset(test_tweet3)
predictions, _ = model_Roberta.predict([test_tweet3])
response_dict = {0: 'Fake', 1: 'Real'}
print("Prediction is: ", response_dict[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Prediction is:  Fake
